In [89]:
import numpy as np

data_path = "/media/ackerman/Windows/Dataset/KITTI/object/training/"
results_detect_path = "results_detect.txt"
results_feature_path = "results_feature.txt"

results_feature_file = open(results_feature_path, "w")
results_feature_file.write("frame\tex\t\t\tey\t\t\tpx\t\t\tpy\t\t\tdensity\t\tintensity\theight\t\tlength\t\twidth\t\torientation\tconfidence\n")

91

In [90]:
def rotate_point(point, rz, center):  
    # rotate axis around z, in center
    rz = -rz
    point[:3] = point[:3] - center
    x = point[0] * np.cos(rz) - point[1] * np.sin(rz)
    y = point[0] * np.sin(rz) + point[1] * np.cos(rz)
    point = np.array([x, y, point[2], point[3]])
    return point

def processPCD(pcd_path, px, py, pz, h, w, l, ry):   
    pcd_data = np.fromfile(pcd_path, dtype=np.float32)  
    pcd_data = pcd_data.reshape(-1, 4)
    rz = -ry

    pcd_data = np.array([rotate_point(point, rz, [px, py, pz]) for point in pcd_data])

    pcd_data = pcd_data[(pcd_data[:, 0] > -w/2) & (pcd_data[:, 0] < w/2) & (pcd_data[:, 1] > -l/2) & (pcd_data[:, 1] < l/2) & (pcd_data[:, 2] > 0) & (pcd_data[:, 2] < h)]

    # pcd density and intensity
    pcd_density = len(pcd_data) / (w * l * h)
    pcd_intensity = np.mean(pcd_data[:, 3])

    return pcd_density, pcd_intensity, pcd_data

In [91]:
with open(results_detect_path, "r") as f:
    results_detect_lines = f.readlines()[1:]
    results_detect_lines = [line.strip().split() for line in results_detect_lines]

for line in results_detect_lines:
    id = int(line[0])
    ex = float(line[1])
    ey = float(line[2])
    px = float(line[3])
    py = float(line[4])
    prob = float(line[5])
    print("processing frame", id)

    with open(data_path + "label_2/" + str(id).zfill(6) + ".txt", "r") as f:
        lines = f.readlines()
        lines = [line.strip().split() for line in lines]
    
    for line in lines:
        if line[0] == "Car" and abs(px-float(line[13]))<0.1 and abs(py+float(line[11]))<0.1:
            pz = -float(line[12])
            orientation = float(line[14])
            direction = float(line[3])
            height = float(line[8])
            width = float(line[9])
            length = float(line[10])

            bin_path = data_path + "velodyne/" + str(id).zfill(6) + ".bin"
            pcd_density, pcd_intensity, _ = processPCD(bin_path, px, py, pz, height, width, length, orientation)
            break
    # check nan
    if np.isnan(pcd_density) or np.isnan(pcd_intensity):
        continue
    
    results_feature_file.write(f"{id}\t\t{ex:.3f}\t\t{ey:.3f}\t\t{px:.3f}\t\t{py:.3f}\t\t{pcd_density:.3f}\t\t{pcd_intensity:.3f}\t\t{height:.3f}\t\t{width:.3f}\t\t{length:.3f}\t\t{orientation:.3f}\t\t{prob:.3f}\n")
    

processing frame 1
processing frame 2
processing frame 3
processing frame 4
processing frame 4
processing frame 6
processing frame 6
processing frame 6
processing frame 6
processing frame 7
processing frame 7
processing frame 7
processing frame 8
processing frame 8
processing frame 8
processing frame 8
processing frame 8
processing frame 8
processing frame 9
processing frame 9
processing frame 10
processing frame 10
processing frame 10
processing frame 10
processing frame 10
processing frame 10
processing frame 10
processing frame 10
processing frame 11
processing frame 11
processing frame 12
processing frame 13
processing frame 15
processing frame 16
processing frame 16
processing frame 16
processing frame 16
processing frame 17
processing frame 18
processing frame 19
processing frame 19
processing frame 20
processing frame 21
processing frame 21
processing frame 21
processing frame 21
processing frame 21
processing frame 21
processing frame 22
processing frame 22
processing frame 23


/home/ackerman/Public/miniconda3/envs/pcdet/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ackerman/Public/miniconda3/envs/pcdet/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


processing frame 96
processing frame 96
processing frame 96
processing frame 96
processing frame 97
processing frame 97
processing frame 98
processing frame 98
processing frame 98
processing frame 98
processing frame 99
processing frame 99
processing frame 99
processing frame 99
processing frame 99
processing frame 99
processing frame 99
processing frame 101
processing frame 101
processing frame 101
processing frame 102
processing frame 102
processing frame 104
processing frame 104
processing frame 104
processing frame 104
processing frame 104
processing frame 105
processing frame 106
processing frame 106
processing frame 106
processing frame 106
processing frame 107
processing frame 107
processing frame 107
processing frame 107
processing frame 107
processing frame 107
processing frame 107
processing frame 108
processing frame 110
processing frame 110
processing frame 110
processing frame 110
processing frame 110
processing frame 110
processing frame 110
processing frame 110
processin

In [94]:
# test
pcd_path = data_path + "velodyne/" + str(6).zfill(6) + ".bin"
x = 19.72
y = 12.54 
z = -1.64
ry = -0.42
rz = -ry
h, w, l = 1.50, 1.62, 3.88

pcd_data = np.fromfile(pcd_path, dtype=np.float32).reshape(-1, 4)
pcd_data = np.array([rotate_point(point, rz, [x, y, z]) for point in pcd_data])
pcd_density, pcd_intensity, pcd_data = processPCD(pcd_path, x, y, z, h, w, l, ry)
print(pcd_density, pcd_intensity)

# x -> w, y -> l, z -> h
# filter points in box
pcd_data = pcd_data[(pcd_data[:, 0] > -w/2) & (pcd_data[:, 0] < w/2) & (pcd_data[:, 1] > -l/2) & (pcd_data[:, 1] < l/2) & (pcd_data[:, 2] > 0) & (pcd_data[:, 2] < h)]


# show pcd
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_data[:, :3])
# show origin and axis
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0, 0, 0])
# o3d.visualization.draw_geometries([pcd, axis])

39.031012685079126 0.06872282587700403
